<a href="https://colab.research.google.com/github/GenaroHacker/actionable_builder/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Database Structure


```
# Table: PLACES
# (ID INTEGER PRIMARY KEY, NAME TEXT UNIQUE)

# Table: CONTEXTS
# (ID INTEGER PRIMARY KEY, PLACE_ID INTEGER, NAME TEXT, RANK INTEGER, FOREIGN KEY(PLACE_ID) REFERENCES PLACES(ID), UNIQUE(PLACE_ID, NAME))

# Table: TAGS
# (ID INTEGER PRIMARY KEY, CONTEXT_ID INTEGER, NAME TEXT, RANK INTEGER, FOREIGN KEY(CONTEXT_ID) REFERENCES CONTEXTS(ID), UNIQUE(CONTEXT_ID, NAME))

# Table: TASKS
# (ID INTEGER PRIMARY KEY, TAG_ID INTEGER, NAME TEXT, RANK INTEGER, FOREIGN KEY(TAG_ID) REFERENCES TAGS(ID), UNIQUE(TAG_ID, NAME))

# Table: STEPS
# (ID INTEGER PRIMARY KEY, TASK_ID INTEGER, NAME TEXT, ORDER_SEQUENCE INTEGER, MINUTES INTEGER, FOREIGN KEY(TASK_ID) REFERENCES TASKS(ID), UNIQUE(TASK_ID, ORDER_SEQUENCE))

# Table: PLACEHOLDERS
# (ID INTEGER PRIMARY KEY, TYPE TEXT, VALUE TEXT, RANK INTEGER, UNIQUE(TYPE, VALUE))

# Table: HISTORY
# (ID INTEGER PRIMARY KEY, STEP_ID INTEGER, ACTION TEXT, TIME TEXT, FOREIGN KEY(STEP_ID) REFERENCES STEPS(ID))

```




In [ ]:
%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/actionable_builder.git


In [ ]:

# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['write_imports', 'actionable_builder']



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world

from actionable_builder.modules.rank_columns import ask_user_cmp
from actionable_builder.modules.rank_columns import ask_user_cmp
from actionable_builder.modules.rank_columns import check_minimum_tasks
from actionable_builder.modules.rank_columns import fetch_records_for_ranking
from actionable_builder.modules.rank_columns import update_rank
from actionable_builder.modules.rank_columns import sort_and_update

from actionable_builder.modules.add_example_data import add_example_data

from actionable_builder.modules.create_database import create_table
from actionable_builder.modules.create_database import insert_record
from actionable_builder.modules.create_database import create_empty_database
from actionable_builder.modules.create_database import check_or_insert_with_parent
from actionable_builder.modules.create_database import check_step_exists
from actionable_builder.modules.create_database import add_task
from actionable_builder.modules.create_database import add_placeholders
from actionable_builder.modules.create_database import initialize_database
from actionable_builder.modules.create_database import create_example_database
from actionable_builder.modules.create_database import download_database

from actionable_builder.modules.sql_core import insert_several_records
from actionable_builder.modules.sql_core import read_records
from actionable_builder.modules.sql_core import read_last_record
from actionable_builder.modules.sql_core import update_record
from actionable_builder.modules.sql_core import remove_record
from actionable_builder.modules.sql_core import run_command

from actionable_builder.modules.pick_zipfian import pick_zipfian





print(write_imports([]))





In [ ]:
# @title Main Execution

# Main Execution
if __name__ == "__main__":
    initialize_database()

    # Example task input
    task = {
        "place": "home",
        "context": "create",
        "tag": "guitar",
        "task": "loop %note%",
        "steps": [
            "identify the note in each string",
            "start the metronome at 30",
            "loop over each string 3 times"
        ]
    }

    add_task(task)

    # Example usage of add_placeholders
    placeholders = {
        "note": ["C", "C#", "D"],
        "color": ["red", "blue", "green"]
    }

    add_placeholders(placeholders)



Choose an option:
1. Create empty database
2. Create database with example data
3. Upload database
2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title Rank Columns

# This script updates ranks in 'tasks.db' tables.
# It groups unranked records by parent ID, sorts them based on user input and updates their ranks.

if __name__ == "__main__":
    db_path = '/content/tasks.db'

    # Sorting and updating ranks for each table, grouped by their parent column
    for table_name, parent_column in [('CONTEXTS', 'PLACE_ID'), ('TAGS', 'CONTEXT_ID'),
                                      ('TASKS', 'TAG_ID'), ('PLACEHOLDERS', 'TYPE')]:
        sort_and_update(db_path, table_name, parent_column)

    download_database()



Sorting for table: CONTEXTS
Sorting for table: TAGS
Sorting for table: TASKS
Sorting for table: PLACEHOLDERS


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>